##### 작성일 : 2021.11.05(금)
##### 작성자  : 손해원

In [3]:
import pandas as pd
from selenium import webdriver
import time
from tqdm import tqdm 

In [4]:
driver = webdriver.Chrome('C:/dev_python/Webdriver/chromedriver') #### 드라이버 경로 설정 

<ipython-input-4-dc79bfd412ee>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/dev_python/Webdriver/chromedriver') #### 드라이버 경로 설정


##### 검색어 설정 

In [5]:
keyword = '김천%20맛집'
url = 'https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate=2020-11-01&endDate=2021-11-05&keyword={keyword}'
driver.get(url)

##### url 수집

In [6]:
### 최근 1년동안의 url만 모으자 
search_number = int(driver.find_element_by_class_name('search_number').text.replace(',','').replace('건','')) # 총 게시물 수 
hrefs = driver.find_elements_by_class_name('desc_inner')
url_list = []
num = 1 

for num in tqdm(range(1, int(search_number/7))):  # 총 게시물 수/7(한 페이지당 게시물 수)
    for i in range(len(hrefs)):
        time.sleep(0.5)
        href = driver.find_elements_by_class_name('desc_inner')[i].get_attribute('href')
        url_list.append(href)
    num += 1
    driver.get(f'https://section.blog.naver.com/Search/Post.naver?pageNo={num}&rangeType=PERIOD&orderBy=sim&startDate=2020-11-01&endDate=2021-11-05&keyword={keyword}')
    
    ##### 에러로 끝나도 당황하지 마세요 !!!!

<ipython-input-6-9da8efb66724>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_number = int(driver.find_element_by_class_name('search_number').text.replace(',','').replace('건','')) # 총 게시물 수
<ipython-input-6-9da8efb66724>:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  hrefs = driver.find_elements_by_class_name('desc_inner')
  0%|                                                                                         | 0/2201 [00:00<?, ?it/s]<ipython-input-6-9da8efb66724>:10: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  href = driver.find_elements_by_class_name('desc_inner')[i].get_attribute('href')
  2%|█▍                                                                            | 39/2201 [03:08<2:54:26,  4.84s/it]


WebDriverException: Message: chrome not reachable
  (Session info: chrome=95.0.4638.69)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B406F3+2492147]
	Ordinal0 [0x00AD9BD1+2071505]
	Ordinal0 [0x009E2340+1057600]
	Ordinal0 [0x009D7474+1012852]
	Ordinal0 [0x009D7C76+1014902]
	Ordinal0 [0x009D9345+1020741]
	Ordinal0 [0x009D3453+996435]
	Ordinal0 [0x009E3690+1062544]
	Ordinal0 [0x00A346D2+1394386]
	Ordinal0 [0x00A2568B+1332875]
	Ordinal0 [0x00A021D4+1188308]
	Ordinal0 [0x00A0302F+1191983]
	GetHandleVerifier [0x00CC67A6+1545030]
	GetHandleVerifier [0x00D7105C+2243580]
	GetHandleVerifier [0x00BCBC97+518199]
	GetHandleVerifier [0x00BCAD80+514336]
	Ordinal0 [0x00ADED2D+2092333]
	Ordinal0 [0x00AE2EE8+2109160]
	Ordinal0 [0x00AE3022+2109474]
	Ordinal0 [0x00AECB71+2149233]
	BaseThreadInitThunk [0x76AB6359+25]
	RtlGetAppContainerNamedObjectPath [0x779087A4+228]
	RtlGetAppContainerNamedObjectPath [0x77908774+180]


In [ ]:
len(url_list)

##### 본문 스크랩

In [ ]:
driver.find_element_by_class_name('')

In [ ]:
title_list = []
writer_list = []
date_list = []
text_list = []
tag_list = []

for i in tqdm(range(len(url_list))):
    driver.get(url_list[i])
    element = driver.find_element_by_id("mainFrame") #iframe 태그 엘리먼트 찾기
    driver.switch_to.frame(element) #프레임 이동
    
    title = driver.find_element_by_class_name('pcol1').text # 제목 
    title_list.append(title)
    writer = driver.find_element_by_class_name('nick').text # 글쓴이
    writer_list.append(writer)
    time.sleep(1)
    try:
        date = driver.find_element_by_class_name('se_publishDate.pcol2').text # 작성일 
        date_list.append(date)
    except:
        date_list.append("NULL")
    try:
        text = driver.find_element_by_class_name('se-main-container').text.replace('\n',' ') # 본문 
        text_list.append(text)
    except:
        text_list.append("NULL")
    
    try: 
        tags = driver.find_elements_by_class_name('item.pcol2.itemTagfont._setTop')
        tags_dummy = []
        for j in range(len(tags)):
            tag = driver.find_elements_by_class_name('item.pcol2.itemTagfont._setTop')[j].text.replace('#','')  # 해쉬태그
            tags_dummy.append(tag)
        tag_list.append(tags_dummy)
    except:
        tag_list.append("NULL")
        
    

In [ ]:
len(title_list)

In [ ]:
len(writer_list)

In [ ]:
len(tag_list)

In [ ]:
len(writer_list)

In [ ]:
len(text_list)

##### 저장

In [ ]:
blog = pd.DataFrame(
    {
        "source":"NAVER_BLOG",
        "url":url_list,
        "title":title_list,
        "writer":writer_list,
        "date":date_list,
        "main_text":text_list,
        "tag":tag_list
    }
)

In [ ]:
blog

In [ ]:
#blog.to_csv('/home/haewon/바탕화면/naver_blog_1109_김천맛집.csv',index=False)